In [63]:
from modules.ONS import queries

index = "ons1516722910092"
store = "ons_featurestore_1516779508509"

boosts = [1.0]*len(models)
queryWeights = [0.5]*len(models)
rescoreWeights = [1.0]*len(models)

size = 10

searchTerms = ["rpi", "gender pay gap", "cpi", "gdp", "inflation", "crime", "unemployment", 
              "population", "immigration", "mental health", "london", "london population", 
              "retail price index", "life expectancy", "obesity", "religion", "migration", 
              "poverty", "social media", "employment"]

qidDict = {}
for i in range(len(searchTerms)):
    qidDict[i+1] = searchTerms[i]

In [21]:
from elasticsearch import Elasticsearch

esUrl = "http://localhost:9200"
esClient = Elasticsearch(esUrl, timeout=1000)

from pymongo import MongoClient, ASCENDING, DESCENDING
mongoClient = MongoClient('localhost', 27017)

db = mongoClient.local
collection = db.judgements

def mergeDicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def termQuery(term):
    return {"term": term}

def timeQuery(dateTime):
    return {"timeStamp": dateTime}

# Get date of most recent entries
doc = collection.find().sort([("timeStamp", DESCENDING)]).limit(1).next()
timeStamp = doc["timeStamp"]
    
timeStampQuery = timeQuery(timeStamp)

In [22]:
import numpy as np

MAX_SCORE = 4.0

def idealJudgement(num):
    i = 0
    incremenet = (1.0 / (float(num) - 1.0)) * num
    
    iJ = np.zeros(num)
    val = len(iJ)
    while (val > 0):
        iJ[i] = (val / float(num)) * MAX_SCORE
        i += 1
        val -= incremenet
        
    return iJ

def idealDiscountedCumulativeGain(num):
    idealGain = idealJudgement(num)
    iDCG = np.zeros(num)
    
    total = 0.0
    for i in range(num):
        total += idealGain[i] / float(i+1)
        iDCG[i] = total
    return iDCG

class Judgements(object):
    def __init__(self, judgements):
        self.judgements = judgements
        
    def dcg(self):
        total = 0.0
        
        dcg = []
        
        for i in range(len(self.judgements)):
            judgement = self.judgements[i]
            total += judgement["judgement"] / float(judgement["rank"])
            dcg.append(total)
            
        return np.array(dcg)
    
    def ndcg(self):
        
        dcg = self.dcg()
        idcg = idealDiscountedCumulativeGain(len(dcg))
        
        ndcg = np.zeros(len(dcg))
        
        for i in range(len(ndcg)):
            ndcg[i] = min(1.0, dcg[i] / idcg[i])
        return ndcg
    
    def __iter__(self):
        return self.judgements.__iter__()
    
    def __getitem__(self, i):
        return self.judgements[i]
    
    def __len__(self):
        return len(self.judgements)
    
    def remove(self, item):
        self.judgements.remove(item)

In [58]:
import copy, json
reload(queries)

def processTerm(searchTerm, rescoreQueries, pages=10):
    mongoQuery = mergeDicts(termQuery(searchTerm), timeStampQuery)
    cursor = collection.find(mongoQuery)
    judgementCount = cursor.count()
    
#     print judgementCount
    
    if (judgementCount > 0):
        judgements = cursor.next()
        modelJudgements = copy.deepcopy(judgements)

        keep = []
        judgementList = modelJudgements["judgements"]["judgementList"]
        
        # Reset ranks
        for judgement in judgementList:
            judgement["rank"] = -1
            
        for page in range(1, pages+1):
            fromParam = (page - 1) * size
            esQuery = queries.getBaseQuery(searchTerm, rescoreQueries, fromParam, size)
#             print json.dumps(esQuery)

            hits = esClient.search(index=index, body=esQuery)
            
#             print "Got %d hits for page %d" % (len(hits["hits"]["hits"]), page)
            
            searchResults = []
            for hit in hits["hits"]["hits"]:
                searchResults.append( hit["_id"] )
            
            # Check for matches
            for judgement in judgementList:
                url = judgement["attrs"]["uri"]
                if (url in searchResults):
                    newRank = int(((page - 1) * 10.0) + searchResults.index(url) + 1)
                    judgement["rank"] = newRank
                    j = copy.deepcopy(judgement)
                    j["rank"] = newRank
                    keep.append(j)
                    
        if (len(keep) > 1):
            judgementList = Judgements(keep)
            ndcg = judgementList.ndcg()
#             print searchTerm, ndcg.mean()
            return ndcg.mean()
    return 0.0
        

In [59]:
# processTerm("rpi", boosts, queryWeights, rescoreWeights)

model = "ons_model_9"
for searchTerm in searchTerms:
    rescoreQuery = queries.getRescoreQueriesForModel(store, searchTerm, model, 1.0, 0.5, 1.0)
    ndcg = processTerm(searchTerm, [rescoreQuery])
    print searchTerm, ndcg

rpi 0.356146510737
gender pay gap 0.381344038499
cpi 0.0
gdp 0.160264666305
inflation 0.0370843059288
crime 0.143854525553
unemployment 0.21582871965
population 0.0
immigration 0.456859258023
mental health 0.512500000497
london 0.0679613965191
london population 0.0570700817232
retail price index 0.01789268271
life expectancy 0.220139297464
obesity 0.0
religion 0.654906862714
migration 0.0497563845439
poverty 0.497389945727
social media 0.0171233752585
employment 0.0


In [77]:
def fn(searchTerm, model, X, doPrint=False):
    b,q,r = X
#     ndcg = processTerm(term, boosts, queryWeights, rescoreWeights)
    rescoreQuery = queries.getRescoreQueriesForModel(store, searchTerm, model,
                        b, q, r)
    ndcg = processTerm(searchTerm, [rescoreQuery])
    if (ndcg is None):
        return 0.0
    return ndcg

def optFn(searchTerm, model, X):
    ndcg = fn(searchTerm, model, X)
    if (ndcg is None):
        return 1.0
    return 1.0 - ndcg

In [104]:
# Individual models
from skopt import gp_minimize

results = {}
# models = ["ons_model_%d" % (i+1) for i in range(9)]
models = ["ons_model_%d" % i for i in [3,9]]

for model in models:
    print "************************************"
    print "model=", model
    print "************************************"
    for searchTerm in searchTerms:
    # searchTerm = "rpi"

        x0 = [(0.0, 1.0), (0.0, 1.0), (0.0, 10.0)]
        termFn = lambda x: optFn(searchTerm, model, x)

        print "Optimizing for term: '%s'" % searchTerm
        res = gp_minimize(termFn, x0, n_calls=20, verbose=False)

        ndcgBefore = fn(searchTerm, model, (1.0, 0.5, 1.0))
        ndcgAfter = fn(searchTerm, model, res.x)
        if (ndcgAfter > ndcgBefore and ndcgAfter > 0.5):
            if (model not in results):
                results[model] = []
            results[model].append(res.x)

        print searchTerm, ndcgBefore, ndcgAfter

************************************
model= ons_model_3
************************************
Optimizing for term: 'rpi'
rpi 0.354292432122 0.707080232398
Optimizing for term: 'gender pay gap'
gender pay gap 0.381040746355 0.382396093272
Optimizing for term: 'cpi'
cpi 0.0734986367799 1.0
Optimizing for term: 'gdp'
gdp 0.16262626861 0.855459211232
Optimizing for term: 'inflation'
inflation 0.0378790390385 0.354376658565
Optimizing for term: 'crime'
crime 0.143218872422 0.747571841401
Optimizing for term: 'unemployment'
unemployment 0.162617662551 0.162617662551
Optimizing for term: 'population'
population 0.0 0.713476287431
Optimizing for term: 'immigration'
immigration 0.456780739083 0.767874542656
Optimizing for term: 'mental health'
mental health 0.501562500062 0.501562500062
Optimizing for term: 'london'
london 0.0476104922472 0.507852506166
Optimizing for term: 'london population'
london population 0.0 0.923068674973
Optimizing for term: 'retail price index'
retail price index 0.037

In [97]:
def fnMultiple(searchTerm, models, X, doPrint=False):
    rescoreQueries = []
    for model,params in zip(models, np.split(np.array(X), len(models))):
        b,q,r = params
#     ndcg = processTerm(term, boosts, queryWeights, rescoreWeights)
        rescoreQuery = queries.getRescoreQueriesForModel(store, searchTerm, model,
                            b, q, r)
        rescoreQueries.append(rescoreQuery)
    ndcg = processTerm(searchTerm, rescoreQueries)
    if (ndcg is None):
        return 0.0
    return ndcg

def optFnMultiple(searchTerm, models, X):
    ndcg = fnMultiple(searchTerm, models, X)
    if (ndcg is None):
        return 1.0
    return 1.0 - ndcg

In [101]:
# Combined models
from skopt import gp_minimize

results = []
# models = ["ons_model_%d" % (i+1) for i in range(9)]
models = ["ons_model_%d" % i for i in [3,9]]

print "************************************"
print "models=", models
print "************************************"
for searchTerm in searchTerms:
# searchTerm = "rpi"

    x0 = [(0.0, 1.0), (0.0, 1.0), (0.0, 10.0)]*len(models)
    termFn = lambda x: optFnMultiple(searchTerm, models, x)
    print "Optimizing for term: '%s'" % searchTerm
    res = gp_minimize(termFn, x0, n_calls=20, verbose=False)

    ndcgBefore = fn(searchTerm, model, (1.0, 0.5, 1.0))
    ndcgAfter = fnMultiple(searchTerm, models, res.x)
    if (ndcgAfter > ndcgBefore):
        results.append(res.x)
    print searchTerm, ndcgBefore, ndcgAfter

************************************
models= ['ons_model_3', 'ons_model_9']
************************************
Optimizing for term: 'rpi'
rpi 0.354260957047 0.767224222721
Optimizing for term: 'gender pay gap'
gender pay gap 0.381042769502 0.381042769502
Optimizing for term: 'cpi'
cpi 0.0 1.0
Optimizing for term: 'gdp'
gdp 0.160267182088 0.855459211232
Optimizing for term: 'inflation'
inflation 0.0362420823471 0.354376658565
Optimizing for term: 'crime'
crime 0.143067110208 0.747571841401
Optimizing for term: 'unemployment'
unemployment 0.215936238385 0.215936238385
Optimizing for term: 'population'
population 0.0 0.713476287431
Optimizing for term: 'immigration'
immigration 0.456859258023 0.767874542656
Optimizing for term: 'mental health'
mental health 0.501562500062 0.434722508183
Optimizing for term: 'london'
london 0.0476104922472 0.507852506166
Optimizing for term: 'london population'
london population 0.0 0.945758380084
Optimizing for term: 'retail price index'
retail price in

In [82]:
for model in models:
    print "************************************"
    print model
    print "************************************"
    for searchTerm in searchTerms:
        sumB = sumQ = sumR = 0.0
        count = 0.0
        if (len(results[model]) > 0):
            for result in results[model]:
                b,q,r = result
                sumB += b
                sumQ += q
                sumR += r
                count += 1.0

    avgB = sumB/count
    avgQ = sumQ/count
    avgR = sumR/count

    print 'b,q,r=', avgB, avgQ, avgR
    print "************************************"
    for searchTerm in searchTerms:
        print searchTerm, fn(searchTerm, model, (1.0, 0.5, 1.0)), fn(searchTerm, model, (avgB, avgQ, avgR))

************************************
ons_model_3
************************************
b,q,r= 0.631887623475 0.125649338855 8.4576390003
************************************
rpi 0.356155262372 0.87665301811
gender pay gap 0.382189500006 0.381482333678
cpi 0.0 0.534620806825
gdp 0.16262626861 0.855457341681
inflation 0.0378723031188 0.354318754639
crime 0.176642282231 0.634061433544
unemployment 0.162617662551 0.162617662551
population 0.0 0.630901365609
immigration 0.456780739083 0.456780739083
mental health 0.512500000497 0.430651997216
london 0.0898870302992 0.50811007846
london population 0.12898119146 0.92225414832
retail price index 0.0415243142619 0.601811239855
life expectancy 0.228727777804 0.623829111225
obesity 0.0 0.0
religion 0.654816960796 0.464688610168
migration 0.0480114556061 0.148777817248
poverty 0.508164996135 0.508164996135
social media 0.0173036505918 0.932409869746
employment 0.0 0.295065124264
************************************
ons_model_9
*********************

In [88]:
reload(queries)

for searchTerm in searchTerms:
    rescoreQueries = []
    
    for model in results:
        sumB = sumQ = sumR = 0.0
        count = 0.0
        if (len(results[model]) > 0):
            for result in results[model]:
                b,q,r = result
                sumB += b
                sumQ += q
                sumR += r
                count += 1.0

        avgB = sumB/count
        avgQ = sumQ/count
        avgR = sumR/count

#         print searchTerm, fn(searchTerm, model, (1.0, 0.5, 1.0)), fn(searchTerm, model, (avgB, avgQ, avgR))
        rescoreQuery = queries.getRescoreQueryForModel(store, searchTerm, model, avgB, avgQ, avgR)
        rescoreQueries.append(rescoreQuery)
        
    ndcgAll = processTerm(searchTerm, rescoreQueries)
    print searchTerm, ndcgAll

rpi 0.767224222721
gender pay gap 0.381042769502
cpi 0.0742005488659
gdp 0.843969137316
inflation 0.0362420823471
crime 0.681805107045
unemployment 0.215936238385
population 0.0260982709755
immigration 0.456859258023
mental health 0.434722508183
london 0.412581172408
london population 0.945758380084
retail price index 0.643158647095
life expectancy 0.569513624567
obesity 0.0
religion 0.703668664481
migration 0.143960006606
poverty 0.497389945727
social media 0.848665100297
employment 0.0802667507106


In [87]:
# def fn(searchTerm, X, doPrint=False):
#     b,q,r = np.split(np.array(X), 3)
#     if (doPrint): print 'b,q,r', b, q, r
# #     ndcg = processTerm(term, boosts, queryWeights, rescoreWeights)
#     ndcg = processTerm(searchTerm, b, q, r)
#     if (ndcg is None):
#         return 0.0
#     return ndcg

# def optFn(searchTerm, X):
#     ndcg = fn(searchTerm, X)
#     if (ndcg is None):
#         return 1.0
#     return 1.0 - ndcg

# standardBQR = np.array([[1.0]*len(models), [0.5]*len(models), [1.0]*len(models)]).flatten()
# ndcgBefore = fn("rpi", standardBQR, doPrint=True)

In [ ]:
# from skopt import gp_minimize

# for searchTerm in searchTerms:
#     print searchTerm
# # searchTerm = "rpi"

#     x0 = [(0.0, 1.0)]*len(models)
#     x0.extend([(0.0, 1.0)]*len(models))
#     x0.extend([(0.0, 10.0)]*len(models))
#     termFn = lambda x: optFn(searchTerm, x)

#     print "Optimizing for term ", searchTerm
#     res = gp_minimize(termFn, x0, n_calls=20, verbose=False)
    
#     ndcgBefore = fn(searchTerm, standardBQR)
#     ndcgAfter = fn(searchTerm, res.x)

#     print searchTerm, ndcgBefore, ndcgAfter

rpi
Optimizing for term  rpi
rpi 0.765749872378 0.761125832596
gender pay gap
Optimizing for term  gender pay gap
gender pay gap 0.381042769502 0.381042769502
cpi
Optimizing for term  cpi
cpi 0.0 0.0742005488659
gdp
Optimizing for term  gdp


In [58]:
for key in resDict:
    res = resDict[key]
#     print res.fun, res.x

    ndcgBefore = fn(searchTerm, [1.0, 0.5, 1.0])
    ndcgAfter = fn(searchTerm, res.x)

    print key, ndcgBefore, ndcgAfter

rpi 0.356146510737 0.767224222721


In [ ]:

[0.91512786228791776, 0.18637723563018363, 0.092858794570568129, 0.53273612668253156,
 0.76706400990704571, 0.58349107122737787, 0.6834580839907477, 0.88558707199299047,
 0.48277903739251804, 6.1817401341518003, 9.7805931268469362, 9.3722047829479767,
 4.713174354828487, 2.601749548125281, 4.3597225473416383, 8.9907343794135848,
 6.4090652493697053, 2.9974113081044855]

Out[53]:
0.76181999486796548